In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split

In [6]:
### load the data

data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
### removing the unesccary data

data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
#### converting gender categorical value into numerical value

label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [9]:
#### converting geography categorical value into numerical value

one_hot_geo=OneHotEncoder()
geo_encoded=one_hot_geo.fit_transform(data[['Geography']])

geo_encoded_df=pd.DataFrame(geo_encoded.toarray(),columns=one_hot_geo.get_feature_names_out(['Geography']))

data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
##### create a pcikle file

with open ('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open ('one_hot_geo.pkl','wb') as file:
    pickle.dump(one_hot_geo,file)



In [11]:
### dependent and independent features

x=data.drop('Exited',axis=1)
y=data['Exited']

In [12]:
#### create the training and test data set

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


### scaler

scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.fit_transform(x_test)



In [14]:
### save the scaler pickle file


with open ('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [17]:
#### ANN CLASSIFICATION


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import pickle
from datetime import datetime

In [18]:
### initialize the sequntial and dense


model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)), ## hl1
    Dense(32,activation='relu'), ## hl2
    Dense(1,activation='sigmoid')
])

model.summary()

c:\Users\user\Desktop\ANN_CLASSIFICATION\env\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
#### create the optimizer

import tensorflow

opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [22]:
#### compile the model


model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [23]:
#### creating logs and saving the logs in the directory


logs= "logs/fit" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=logs,histogram_freq=1)


In [24]:
### earlystopping callback

earlystopping_callback=EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)


In [25]:
#### training the model

model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,earlystopping_callback]
)

model.save("model.h5")

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.8346 - loss: 0.3971 - val_accuracy: 0.8535 - val_loss: 0.3636
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8500 - loss: 0.3598 - val_accuracy: 0.8515 - val_loss: 0.3489
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8553 - loss: 0.3509 - val_accuracy: 0.8540 - val_loss: 0.3507
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8559 - loss: 0.3464 - val_accuracy: 0.8555 - val_loss: 0.3453
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8599 - loss: 0.3419 - val_accuracy: 0.8580 - val_loss: 0.3512
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8604 - loss: 0.3393 - val_accuracy: 0.8560 - val_loss: 0.3432
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8609 - loss: 0.3362 - val_accuracy: 0.8630 - val_loss: 0.3393
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8616 - loss: 0.3363 - val_accu

In [27]:
### load extension board

%load_ext tensorboard

In [ ]:
### launching the tensorboard

%tensorboard --logdir logs/fit20260207-200307 

Reusing TensorBoard on port 6006 (pid 3880), started 0:00:48 ago. (Use '!kill 3880' to kill it.)